# Scraping Hotel Ratings on Booking # 

In this homework we will practice web scraping on the following [site](https://www.booking.com/searchresults.html?aid=304142&label=gen173nr-1DCAEoggJCAlhYSDNiBW5vcmVmcgV1c19tYYgBAZgBMcIBA2FibsgBDdgBA-gBAfgBApICAXmoAgQ&sid=28d97f630803f9d48b4a1f535cbdd33f&class_interval=1&dest_id=20061717&dest_type=city&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=city&room1=A%2CA&sb_price_type=total&src=index&src_elem=sb&ss=Boston&ssb=empty&ssne_untouched=Cancún&rows=15). Let's get some basic information for each hotel in Boston.
On each hotel page, scrape the following information: 
1. Hotel Name
2. Class of Rating (Wonderful/Excellent/Very Good/Good)
3. Rating Score
4. Number of Reviews


** Save the data in "traveler_ratings.csv" in the following format: hotel_name, class_of_rating, rating, num_reviews **

**(10 pts)**

You can see an overview of the information as displayed:





![Information to be scraped](booking_sample.png)

In [2]:
from bs4 import BeautifulSoup
import sys
import time
import os
import logging
import requests
import codecs
import json
import re

base_url = "http://www.booking.com"
#Describes your browser
user_agent = "Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36"

datadir = "data"
city = "Boston"
hotel_links = []
outfile = open('traveler_ratings.csv', 'w')
outfile.write('hotel_name, class_of_rating, rating, num_reviews\n')

log = logging.getLogger(__name__)
log.setLevel(logging.ERROR)
loghandler = logging.StreamHandler(sys.stderr)
loghandler.setFormatter(logging.Formatter("[%(asctime)s] %(message)s"))
log.addHandler(loghandler)


def get_city_page(city):
    url = base_url + "/" + city
    log.info("URL TO REQUEST: %s \n" % url)
    headers = {'User-Agent' : user_agent}
    response = requests.get(url, headers=headers)
    html = response.text.encode('utf-8')
    
    with open(os.path.join(city + '-search-page.json'), "w") as h:
        h.write(html.decode("utf-8").encode('cp850', 'replace').decode('cp850'))

    soup = BeautifulSoup(html, "html5lib")
    li = soup.findAll('a', href=True)
    for el in li:
        if el.find(text=re.compile('All')) and el.find(text=re.compile('properties in')):
            return(el['href'])

    log.info("City not listed")

    
    
def get_hotellist_page(city_url, count):
    url = base_url + city_url
    
    time.sleep(2)
    
    headers = { 'User-Agent' : user_agent }
    response = requests.get(url, headers=headers)
    html = response.text.encode('utf-8')
    
    with open(os.path.join(datadir, city + '-hotellist-' + str(count) + '.html'), "w") as h:
        h.write(html.decode("utf-8").encode('cp850', 'replace').decode('cp850'))

    return(html)



def parse_hotellist_page(html, hotel_links):
    soup = BeautifulSoup(html, "html5lib")
 
    hotel_boxes = soup.findAll('div', {'class' :re.compile('sr_item_default')})
    
    for hotel_box in hotel_boxes:
        
        state = True
        try:
            hotel_name = hotel_box.find('span', {'class' : 'sr-hotel__name'}).find(text=True).strip()
            link = hotel_box.find('a', {'class' :'hotel_name_link url'})['href'].strip()
            
        except Exception:
            state = False
            log.error("Hotel not found")
            hotel_name = ""
            class_of_rating = "N/A"
            rating = "N/A"
            num_reviews = "N/A"
            link = ""
        
        if (state):
            try:
                class_of_rating = hotel_box.find('span', {'class' : 'review-score-widget__text'}).find(text=True).strip()
                rating = hotel_box.find('span', {'class' :'review-score-badge'}).find(text=True).strip()  
                num_reviews = hotel_box.find('span', {'class' : 'review-score-widget__subtext'}).find(text=True).strip()
            
            except Exception:
                class_of_rating = "N/A"
                rating = "N/A"         
                num_reviews = "N/A"
                
        log.info("HOTEL NAME: %s" % hotel_name)
        log.info("HOTEL REVIEWS: %s" % num_reviews)
        log.info("HOTEL STAR RATING: %s " % rating)
        log.info("HOTEL CLASS OF RATINGS: %s \n" % class_of_rating)
        
        hotel_links += [link]
        line = hotel_name.replace(',', '') + ',' + class_of_rating + ',' + rating + ',' + num_reviews.replace(',', '') + '\n'
        outfile.write(line)
        
    div = soup.find("div", {"class" : "results-paging"})
    if div.find('span', {'class':'paging-end'}):
        log.info('We reached last page')
        return False
    hrefs = div.findAll('a', href=True)
    
    for href in hrefs:
        if href.find(text=True) == 'Next page':
            log.info("Next url is %s" % href['href'])
            return href['href']
        
city_url = get_city_page(city)

c = 0
page_urls = []
while(city_url):
    c += 1
    h_page = get_hotellist_page(city_url, c)
    city_url = parse_hotellist_page(h_page, hotel_links)
    page_urls += [city_url]
outfile.close()
print("Look at csv file")

Look at csv file


Now let's scrape some reviews. For each review of each each hotel in Boston you are to scrape the following attributes: 
1. Reviewer name
2. Reviewer ethnicity
3. Number of reviews 
4. Number of helpful votes
5. Date
6. Rating
7. Negative Review
8. Positive Review

Note that you will also need the hotel's name!! Also, some reviews may not have all attributes. 

** Save the data in "review_ratings.csv" in the following format: hotel_name, reviewer_name, ethnicity, num_reviews, num_help_votes, date, rating, neg_review, pos_review **

**(25 pts)**

You can see an overview of the information as displayed:
![Information to be scraped](review_sample.png)

In [3]:
review_links = []
block_display = []
exp = []
reviews_outfile = open("review_ratings.csv", "w", encoding="utf-8")
reviews_outfile.write("Hotel Name, Reviewer Name, Reviewer Ethnicity, Number of Reviews, Date, Rating, Positive ReviewsS, Negative Reviews\n")

def get_review_links(hotel_links, review_links, block_display): 
    c = 0
    for hotel_url in hotel_links:
        c += 1
        print(c)
        url = base_url + hotel_url

        time.sleep(2)

        headers = { 'User-Agent' : user_agent }
        response = requests.get(url, headers=headers)
        html = response.text.encode('utf-8')
        soup = BeautifulSoup(html, "html5lib")
                
        
        try:
            review_link = soup.find('a', {'class' : 'show_all_reviews_btn'})['href']
    
        except Exception:
            block_display += [url]
            
        if (review_link == "#blockdisplay4"):
            block_display += [url]
        else:
            review_links += [review_link]
            

            
get_review_links(hotel_links, review_links, block_display)
print("got links")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [6]:
def extract_link_reviews(review_links):
    for rv_link in review_links:

        url = base_url + rv_link
        
        time.sleep(3)
        
        headers = {'User-Agent' : user_agent }
        response = requests.get(url, headers=headers)
        html = response.text.encode('utf-8')
        soup = BeautifulSoup(html, "html5lib")
        
        ht_name = soup.find('a', {'class' : 'standalone_header_hotel_link'}).text.strip()
        
        
        review_box = soup.findAll('li', {'class':'review_item clearfix '})
        
        for review in review_box:
            try:
                reviewer_name = review.find('span', {'itemprop' : 'name'}).find(text=True).strip()
                reviewer_ethnicity = review.find('span', {'itemprop' : 'nationality'}).text.strip()
                reviewer_num_reviews = review.find('div', {'class':'review_item_user_review_count'}).text.strip()
                review_date = review.find('p', {'class':'review_item_date'}).text.strip()
                review_rating = review.find('span', {'class':'review-score-badge'}).text.strip()
                
            
            except Exception:
                reviewer_name = "N/A"
                reviewer_ethnicity = "N/A"
                reviewer_num_reviews = "N/A"
                review_date = "N/A"
                review_rating = "N/A"
            try:
                neg_review = review.find('p', {'class':'review_neg'}).text.strip()
                pos_review = review.find('p', {'class' : 'review_pos'}).text.strip()
                
            except Exception:
                neg_review = "N/A"
                pos_review = "N/A"
                
            line = ht_name.replace(',', '')+','+reviewer_name+','+reviewer_ethnicity+','+reviewer_num_reviews+','+review_date+','+review_rating+','+pos_review.replace('\n', '')+','+neg_review.replace('\n', '') + '\n'
            reviews_outfile.write(line)
            
if (review_links != 0):
    extract_link_reviews(review_links)
else:
    print("no links")

reviews_outfile.close()
print("Done")


Done


The reviews that were collected are the reviews of the hotels that have separate review pages as opposed to modal boxes.
When trying to access the reviews from the modal boxes BeautifulSoup was not about the find the <li> tags that contained the information or even the <ul> tag that contained the <li> tags for the reviews.
Thus in my code, I have separated the hotels by those that have the actual review page (review links) and those that don't and instead have the modal boxes (block_display)